In [ ]:
# If running from Colab, please uncomment and run the following cell to get summer installed.
#!pip install summerepi2==1.3.5

In [ ]:
import pandas as pd
from datetime import datetime
from summer2 import CompartmentalModel
from summer2.parameters import Parameter
from plotly import graph_objects as go
pd.options.plotting.backend = 'plotly'

## Epidemiological explanation
Here we take a similar epidemiological model to that presented in 
the seir_model.ipynb notebook and replicate the latent compartments in series.
### Flow adjustments
In order to interpose additional latent or exposed compartments
between the susceptible and infectious compartments,
we need to make four structural adjustments to the model, as follows:
- We remove the previous process of infection transfering newly infected persons to the single exposed compartment
- The infection process needs to transfer newly infected person to the first of the sequential exposed compartments
- We add transitions between each of the sequential latent compartments
- We add a transition from the last of the latent compartments to the infectious compartment

### Transition rates and parameters
We can retain the same parameters as in for the base SEIR model.
However, this will result in much slower rates of arrival in the infectious compartment following infection
if we retain the same rate parameter for each of the transfer processes.
This is because infected persons will need to transition between each of the sequential compartments,
and so only arrive in the infectious compartment once they complete these transition processes.

We can offset this by multiplying the rate of each transition by the number of transitions that must be made.
This will result in the same mean time from infection to onset of infectiousness,
such that we can still think of the parameter as the average incubation period.
However, this will have implications for model dynamics,
because the distribution of arrivals into the infectious compartment will differ from that under the base SEIR model.
Specifically, the base case implies an exponential distribution for this time,
whereas this model configuration implies an Erlang-distributed arrival time with shape parameter
equal to the number of sequential compartments.

In [ ]:
iso = 'MYS'

In [ ]:
cases_data = pd.read_csv('https://github.com/monash-emu/wpro_working/raw/main/data/new_cases.csv', index_col=0)[iso]
cases_data.index = pd.to_datetime(cases_data.index)
approx_pops = {
    'MYS': 33e6,
    'PHL': 114e6,
    'VNM': 97e6,
}

In [ ]:
n_latent_comps = 3
latent_comps = [f'exposed_{i_comp}' for i_comp in range(n_latent_comps)]
compartments = ['susceptible', 'infectious', 'recovered'] + latent_comps

In [ ]:
analysis_start_date = datetime(2021, 8, 1) 
analysis_end_date = datetime(2022, 6, 1)
compartments = ['susceptible', 'infectious', 'recovered'] + latent_comps
epi_model = CompartmentalModel(
    [analysis_start_date, analysis_end_date],
    compartments,
    ['infectious'],
    ref_date=datetime(2019, 12, 31),
)

In [ ]:
# Transition flows adjusted relative to the base SEIR model
epi_model.add_infection_frequency_flow('infection', Parameter('contact_rate'), 'susceptible', latent_comps[0])
for i_comp in range(n_latent_comps - 1):
    source_comp = f'exposed_{i_comp}'
    dest_comp = f'exposed_{i_comp + 1}'
    transfer_rate = float(n_latent_comps) / Parameter('incubation_period')
    epi_model.add_transition_flow(f'exposed_transfer_{i_comp}', float(n_latent_comps) / Parameter('incubation_period'), source_comp, dest_comp)
epi_model.add_transition_flow('progression', float(n_latent_comps) / Parameter('incubation_period'), latent_comps[-1], 'infectious')

In [ ]:
epi_model.add_transition_flow('recovery', 1.0 / Parameter('infectious_period'), 'infectious', 'recovered')
epi_model.set_initial_population({'susceptible': approx_pops[iso], 'exposed_0': 1.0})
incidence = epi_model.request_output_for_flow('incidence', 'progression')
epi_model.request_function_output('notifications', incidence * Parameter('detection_prop'));

In [ ]:
parameters = {
    'contact_rate': 0.4,
    'incubation_period': 5.0,
    'infectious_period': 5.0,
    'detection_prop': 0.07,
}
epi_model.run(parameters)

In [ ]:
plot_start_date = datetime(2021, 10, 1)
comparison_plot = epi_model.get_derived_outputs_df()["notifications"].plot()
comparison_plot.add_trace(go.Scatter(x=cases_data.index, y=cases_data))
comparison_plot.update_xaxes(range=(plot_start_date, analysis_end_date))

In [ ]:
epi_model.run(parameters | {"contact_rate": 0.0})
epi_model.get_derived_outputs_df()["incidence"].iloc[:30].plot()